In [8]:
import pandas as pd
import os, glob
import time
import numpy as np
import struct

begining_start_time = time.time()

folder_locations = [r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments01",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments02",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments03",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments06",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments07",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments08",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments12",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments13",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments14",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments15",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments16",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments17",
                    r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\IndividualProject_Bisal_Datasets\Dataset\PersonSitting\experiments18"]



final_dataframe = pd.DataFrame()

for folder in folder_locations:
    start_time = time.time()
    path = os.path.join(folder, "binaries")

    intermediate_df = pd.DataFrame()

    for filename in glob.glob(os.path.join(path, '*.npy')):
        with open(filename, 'rb') as f:
            data = np.load(f)
            data_bytes = data.tobytes()

            ultrasonic_data = []
            for i in struct.iter_unpack('@h', data_bytes[64:]):
                ultrasonic_data.append(i[0])

            tmp_df = pd.DataFrame([ultrasonic_data])
            label_output = os.path.splitext(os.path.basename(filename))[0].split("_")[2]

            if label_output.lower() == "p":
                label_output = "1"
            else:
                label_output = "0"

            tmp_df.insert(0, "label", [label_output,])

            intermediate_df = pd.concat([intermediate_df, tmp_df])

    final_dataframe = pd.concat([final_dataframe, intermediate_df])
    print(time.time() - start_time)

final_dataframe.to_csv("Person.csv", index=False, header=None, sep=',')

print(time.time() - begining_start_time)


21.9592866897583
55.73212790489197
74.38191652297974
55.7223334312439
15.983967781066895
23.72145652770996
80.07549715042114
90.9430501461029
40.18657040596008
110.29175329208374
13.12543272972107
11.665015935897827
23.963396787643433
619.2283341884613


In [9]:
data = pd.read_csv("Person.csv", header=None)
data.shape


(41901, 86)

In [ ]:
import pandas as pd
import os
import numpy as np
import struct

folder_path = r"C:\Users\yousuf\source\repos\individual_project_data_label\Program\python\person-detection\experiments"
binaries_path = os.path.join(folder_path, "binaries")

output_data = []

for filename in os.listdir(binaries_path):
    if filename.endswith(".npy"):
        file_path = os.path.join(binaries_path, filename)
        label = os.path.splitext(filename)[0].split("_")[2]

        if label.lower() == "p":
            label = "person"
        else:
            label = "non_person"

        data = np.load(file_path)
        ultrasonic_data = struct.unpack(f'{len(data)//2}h', data[64:])

        row = [label] + list(ultrasonic_data)
        output_data.append(row)

output_columns = ["label"] + [f"ultrasonic_{i}" for i in range(len(output_data[0]) - 1)]
output_df = pd.DataFrame(output_data, columns=output_columns)

output_csv_path = os.path.join(folder_path, "PersonMeasurements.csv")
output_df.to_csv(output_csv_path, index=False)
